In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import tensorflow as tf
import tensorflow.keras.backend as K

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity

### Helping functions

In [2]:
def get_dataset(path='data/emotions.csv'):
    return pd.read_csv(path)

def get_all_au(df):
    return df.loc[:, ' AU01_c':' AU45_c']

def get_intensity_au(df):
    return df.loc[:, ' AU01_r':' AU45_r']

def get_presense_au(df):
    return df.loc[:, ' AU01_c':' AU45_c']

def get_facial_landmarks(df):
    return df.loc[:, ' x_0':' y_67']

def get_au_facial_landmarks(df):
    X_l = df.loc[:, ' x_0':' y_67']
    X_a = df.loc[:, ' AU01_r':' AU45_c']
    X = pd.concat([X_l, X_a], axis=1)
    return X

#### Taken from old keras source code

In [3]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

# Test different method for classification for maximum emotion


### Load  and split data for test and train

In [4]:
df = get_dataset('data/emotions.csv')
X = get_facial_landmarks(df)
y = y = df.emotion_label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = np.asarray(X_train), np.asarray(X_test)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)
y_test = np.argmax(y_test, axis=1)

### (1) Random forest

#### Set parameters

In [5]:
n_estimators = [5,  7, 9, 16, 17, 22, 44]
max_depth = [5, 10, 15, 20, 25, 30, 35]
criterion = ['gini', 'entropy']

#### Test different configurations

In [6]:
max_f1 = 0.0
max_param = ""

for estimator in tqdm(n_estimators):
    for depth in max_depth:
        for crit in criterion:
            param = str(estimator) + str(depth) + str(crit)
            clf = RandomForestClassifier(n_estimators=estimator,
                                        max_depth = depth,
                                        criterion=crit)                                        
            clf.fit(X_train, y_train)
            y_pred = np.argmax(clf.predict(X_test), axis=1)
            au_random_forest_f1_test = metrics.f1_score(y_test, y_pred, average='macro')
            
            if au_random_forest_f1_test > max_f1:
                max_f1 = au_random_forest_f1_test
                max_param = param

100%|██████████| 7/7 [10:59<00:00, 94.19s/it] 


### (2) Neural networks

#### Set parameters

In [7]:
learning_rates = [0.01, 0.001, 0.0001, 0.00001]
losses = ['binary_crossentropy', tf.keras.losses.mean_squared_error]
optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop]
input_len = len(X_train[1])

#### Test different configurations

In [8]:
max_param = ""
max_f1 = 0

for learning_rate in tqdm(learning_rates):
    for loss in losses:
         for optimizer in optimizers:
                param = str(learning_rate) + str(loss) + str(optimizer)
                model = Sequential()
                model.add(Dense(input_len, input_dim=input_len, activation='relu'))
                model.add(Dense(8, activation='sigmoid'))
                opt = optimizer(learning_rate=learning_rate)
                model.compile(loss=loss, optimizer=opt, metrics=['accuracy', get_f1])
                history = model.fit(X_train, y_train , epochs=50, verbose=0)
                y_pred = np.argmax(model.predict(X_test), axis=1)
                y_pred = np.argmax(model.predict(X_test), axis=1)
                au_model_f1_test = metrics.f1_score(y_test, y_pred, average='macro')
                
                if max_f1 < au_model_f1_test:
                    max_f1 = au_model_f1_test
                    max_param = param

100%|██████████| 4/4 [08:53<00:00, 133.38s/it]


# Test different method for classification for emotion vector 

### Load  and split data for test and train

In [9]:
df = get_dataset('data/emotions.csv')
X = get_facial_landmarks(df)
y = df.emotion_label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = np.asarray(X_train), np.asarray(X_test)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

### (1) Random Forest

#### Set parameters

In [10]:
n_estimators = [5,  7, 9, 16, 17, 22, 44]
max_depth = [5, 10, 15, 20, 25, 30, 35]
criterion = ['gini', 'entropy']

#### Test different configurations

In [11]:
max_similarity = 0
max_param = ''

for estimator in tqdm(n_estimators):
    for depth in max_depth:
        for crit in criterion:
            param = str(estimator)+str(depth)+str(crit)
            clf = RandomForestClassifier(n_estimators=estimator,
                                        max_depth = depth,
                                        criterion=crit)                                        
            clf.fit(X_train, y_train)
            y_pred =  clf.predict(X_test)
            cosine_similarity_arr = cosine_similarity(y_pred, y_test)
            counter = 0
            for i in range(len(y_pred)):
                if cosine_similarity_arr[i][i] >= 0.6:
                    counter = counter +1 
            tmp = counter / len(y_pred) * 100
            if tmp > max_similarity:
                max_similarity = tmp
                max_param = param

100%|██████████| 7/7 [10:30<00:00, 90.07s/it] 


### (2) Neural networks

#### Set parameters

In [12]:
learning_rates = [0.01, 0.001, 0.0001, 0.00001]
losses = ['binary_crossentropy', tf.keras.losses.mean_squared_error]
optimizers = [tf.keras.optimizers.Adam, tf.keras.optimizers.RMSprop]
input_len = len(X_train[1])
y_test = np.argmax(y_test, axis=1)

#### Test different configurations

In [13]:
max_param = ""
max_f1 = 0
for learning_rate in tqdm(learning_rates):
    for loss in losses:
         for optimizer in optimizers:
                param = str(learning_rate) + str(loss) + str(optimizer)
                model = Sequential()
                model.add(Dense(input_len, input_dim=input_len, activation='relu'))
                model.add(Dense(8, activation='sigmoid'))
                opt = optimizer(learning_rate=learning_rate)
                model.compile(loss=loss, optimizer=opt, metrics=['accuracy', get_f1])
                history = model.fit(X_train, y_train , epochs=50, verbose=0)
                y_pred = np.argmax(model.predict(X_test), axis=1)
                au_model_f1_test = metrics.f1_score(y_test, y_pred, average='macro')

                if max_f1 < au_model_f1_test:
                    max_f1 = au_model_f1_test
                    max_param = param

100%|██████████| 4/4 [08:45<00:00, 131.38s/it]
